In [1]:
import pandas as pd

 Volume in drive C has no label.
 Volume Serial Number is BA7B-ACA1

 Directory of C:\Users\hp\Desktop\RedBuffer\Gold_Prices_task_1\data

04/19/2022  03:33 AM    <DIR>          .
04/19/2022  03:33 AM    <DIR>          ..
04/19/2022  03:29 AM         6,088,022 Prices.xlsx
               1 File(s)      6,088,022 bytes
               2 Dir(s)  55,316,267,008 bytes free


In [25]:
df = pd.read_excel('../data/Prices.xlsx', sheet_name='Daily_Indexed', skiprows=8, usecols='D:E')

In [26]:
df.head()

,Name,US dollar
0,1978-12-29,78.53
1,1979-01-01,78.53
2,1979-01-02,78.80
3,1979-01-03,75.96
4,1979-01-04,77.54
